In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [4]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [5]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "qwiklabs-gcp-02-dd1ad620be77"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [4]:
model = GenerativeModel("gemini-1.5-flash")

In [6]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [7]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some name ideas for a flower shop specializing in dried flowers, playing with different themes and styles:

**Whimsical & Romantic:**

* Everlasting Blooms
* Dried & Delightful
* The Petal Press
* Whispers of Wildflowers
* Bloom & Remember
* The Sun-Kissed Bouquet
* The Dusty Rose

**Modern & Minimalist:**

* Bloom & Co.
* The Dried Flower Bar
* Paper & Petal
* Botanicals & Co.
* The Dried Bouquet
* Bloom Forever
* The Lasting Petal

**Nature & Earthy:**

* Wildflower Gatherings
* Meadow & Bloom
* The Wildflower Press
* The Dried Garden
* Whispering Pines
* Forest & Bloom

**Unique & Playful:**

* The Flower Whisperer
* The Boho Bouquet
* Dried & Dusted
* The Forever Bloom
* The Petal Post
* The Botanical Muse
* The Sun-Kissed Bloom

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Reflect your brand aesthetic.**
* **Check availability of the name and domain name.**
* **Get feedback from friends and potential customers.** 

**Bonus:**

* Consider using you

✅ Recommended. The prompt below is to the point and concise.

In [8]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Flower Shop Names for Dried Bouquets:

**Evoking Timeless Beauty and Nostalgia:**

* **The Dried Bloom**
* **Everlasting Blooms**
* **Whispers of Time**
* **Bloom & Dusk**
* **Faded Glory**
* **Dried & Divine**
* **The Botanical Archive**
* **The Paper Petal**
* **Forever in Bloom**
* **Timeless Blooms**

**Highlighting the Natural and Sustainable Aspect:**

* **Earth & Bloom**
* **The Dried Garden**
* **Natural Beauty**
* **Botanical Treasures**
* **Sustainable Blooms**
* **The Dried Petal Project**
* **The Meadow & Bloom**
* **The Eco-Bloom**
* **The Botanical Haven**

**Playful & Unique:**

* **Bloom & Dust**
* **The Dried Flower Co.**
* **Petal & Twig**
* **The Dusty Bouquet**
* **The Dried Bloom Bar**
* **Blooms & Beyond**
* **The Flower Alchemist**

**Consider including your location or a personal touch:**

* **[Your Name]'s Dried Blooms**
* **[City] Dried Flowers**
* **The [Neighborhood] Dried Bouquet Shop**

**Tips for Choosing a Name:**

* **Keep it short and easy to rememb

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [9]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known celestial body to harbor life. It's a dynamic and complex world with a rich history and an uncertain future. Here's a glimpse into some of Earth's defining features:

**Physical Characteristics:**

* **Size and Shape:** Earth is a slightly flattened sphere with a diameter of approximately 12,742 kilometers. 
* **Composition:** Mostly iron, nickel, silicon, oxygen, magnesium, sulfur, and aluminum.
* **Atmosphere:** Primarily nitrogen (78%) and oxygen (21%), with trace amounts of other gases. The atmosphere protects us from harmful radiation and regulates Earth's temperature.
* **Surface:** Diverse and dynamic, featuring continents, oceans, mountains, valleys, glaciers, and deserts.
* **Hydrosphere:** 71% of Earth's surface is covered in water, primarily saltwater in oceans.
* **Climate:** Varies greatly from region to region, influenced by latitude, altitude, and ocean currents. 
* **Magnetic Field:** G

✅ Recommended. The prompt below is specific and well-defined.

In [10]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What Makes Earth Unique:

**1. Abundant Liquid Water:** Earth is the only planet in our solar system with liquid water on its surface, covering 71% of the planet. This is crucial for life as we know it.

**2. Plate Tectonics:** Earth's unique geological activity driven by plate tectonics creates mountains, volcanoes, and earthquakes, constantly reshaping the planet's surface and driving its climate.

**3. Atmosphere with Oxygen:** Earth's atmosphere is rich in oxygen, produced by photosynthetic life. This oxygen is essential for animal respiration and the development of complex life forms.

**4. Strong Magnetic Field:** Earth's magnetic field acts as a shield against harmful solar radiation, protecting life on the surface.

**5. Stable Climate:** Despite variations, Earth enjoys a relatively stable climate thanks to a delicate balance of greenhouse gases, ocean currents, and atmospheric circulation.

**6. Diverse Life Forms:**  Earth hosts an astonishing diversity of life, from micr

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [11]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Here's a breakdown of those two questions:

**Boiling Water**

There isn't one single "best" method, but here are some effective ones:

* **Stovetop:** This is the most common and straightforward way. Simply fill a pot with water and heat it on a stovetop burner. This is efficient and works well for most purposes. 
* **Electric Kettle:** Electric kettles are designed specifically for boiling water quickly. They are convenient and often energy-efficient.
* **Microwave:** You can boil water in a microwave-safe container, though it can be less efficient and may not be suitable for large volumes.
* **Camping Stove:** For outdoor use, a camping stove with a pot can efficiently boil water.

**Why is the Sky Blue?**

This has to do with how sunlight interacts with the Earth's atmosphere. Here's the simplified explanation:

1. **Sunlight:** The sun emits a spectrum of colors (like a rainbow).
2. **Scattering:** When sunlight enters the Earth's atmosphere, it collides with tiny molecules like n

✅ Recommended. The prompts below asks one task a time.

In [13]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There isn't really a "best" method of boiling water, as it depends on your needs and preferences. However, here are some factors to consider:

**1.  Speed:**

* **Microwave:** Fastest method, especially for small amounts of water. 
* **Electric kettle:** Also very fast, and often safer than stovetop.
* **Stovetop:** Slower than microwave or electric kettle, but more versatile for larger volumes. 

**2.  Safety:**

* **Electric kettle:** Generally considered the safest option, with built-in safety features like auto-shutoff.
* **Microwave:** Can be dangerous if the container isn't microwave-safe, or if you forget about it. 
* **Stovetop:** Requires constant supervision and caution to avoid burns.

**3.  Convenience:**

* **Electric kettle:** Easy to use and clean, and often has temperature control features.
* **Microwave:** Requires less space and can be used for other tasks.
* **Stovetop:** Requires a pot and burner, which can be less convenient.

**4.  Energy efficiency:**

* **Electr

In [14]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky is blue due to a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is made up of all colors of the rainbow.** When sunlight enters the Earth's atmosphere, it interacts with the tiny molecules of air, primarily nitrogen and oxygen.
* **These molecules are much smaller than the wavelengths of visible light.**  This size difference is crucial.
* **Shorter wavelengths of light (like blue and violet) scatter more easily than longer wavelengths (like red and orange).** This is why the sky appears blue during the day. The blue light is scattered in all directions, making the sky appear blue.

**Here's an analogy:** Imagine sunlight as a stream of tiny balls of different colors. When the balls hit the air molecules, the blue and violet balls bounce off in all directions more often than the red and orange balls.

**Why isn't the sky violet?** 

While violet light scatters even more than blue, our eyes are more sensitive to blue light. So, we perceive the scat

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [15]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is today, you can:

* **Check your phone or computer:** Most devices have a built-in clock that displays the current date.
* **Ask a voice assistant:** If you have a smart speaker or phone, you can ask "What day is it today?"
* **Look at a calendar:** You can find a calendar online or in a physical format. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [16]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan offers a wealth of sightseeing opportunities! To best recommend a place for you, I need a bit more information. What kind of sights are you interested in? 

* **Art and History:**  Are you looking for famous museums like the Duomo or the Pinacoteca di Brera, or perhaps Renaissance architecture like the Sforza Castle?
* **Shopping and Fashion:** Do you want to explore the luxury boutiques on Via Montenapoleone or browse the stylish shops in the Navigli district?
* **Food and Culture:** Are you interested in sampling traditional Milanese cuisine or experiencing the city's vibrant nightlife?

Once you tell me what you're most interested in, I can give you a more tailored recommendation. 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [17]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers with their trip planning, not provide personal recommendations for dinner.  

However, I can help you find great restaurants in Milan! Tell me what kind of cuisine you're interested in, and your budget, and I'll be happy to suggest some options. 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [18]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

## Programming Activity Recommendations for High School Students:

**Level: Beginner**

* **Build a simple text-based game:** Choose a classic like Hangman or Guess the Number, and focus on learning basic input/output, loops, and conditional statements.
* **Create a website using HTML, CSS, and JavaScript:** Start with simple pages like a portfolio or a blog. Experiment with styling, animations, and interactivity.
* **Learn Python and build a basic calculator or unit converter:** Python is beginner-friendly and has many libraries for common tasks.
* **Contribute to an open-source project:** Find a project on GitHub related to your interests and offer to help with small tasks like fixing bugs or adding features.

**Level: Intermediate**

* **Develop a simple mobile app using MIT App Inventor or Thunkable:** Create a fun app with basic features like user input, data storage, and notifications.
* **Learn data structures and algorithms:** Build a sorting algorithm (e.g., bubble sort) or im

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [19]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

As a high school student, you're in a fantastic position to learn programming. All three options are valuable, but the best choice depends on your interests and goals:

**a) Learn Python:**

* **Pros:** 
    * **Beginner-friendly:** Python's syntax is clear and readable, making it a great language for beginners.
    * **Versatile:** Python is used in web development, data science, machine learning, automation, and more.
    * **Large community and resources:**  A huge community provides plenty of tutorials, documentation, and support.
* **Cons:**
    * **Not as performant for certain tasks:** While versatile, Python might not be the best choice for high-performance applications. 

**b) Learn JavaScript:**

* **Pros:**
    * **Essential for web development:** JavaScript is the language that makes websites interactive. 
    * **In-demand:**  JavaScript skills are highly sought after in the tech industry.
    * **Growing ecosystem:**  JavaScript is used in a wide range of applications bey

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [20]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [21]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [22]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 

Here's why:

* **"Something surprised me about this video - it was actually original."** This indicates the tweet author found the video refreshing and different, which is a positive sentiment.
* **"It was not the same old recycled stuff that I always see."**  This reinforces the positive aspect of the video being original.
* **"Watch it - you will not regret it."** This is a strong recommendation, indicating a positive opinion. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.